# Exercícios – Semana 4: Probabilidade e Estatística

Este notebook contém soluções para os exercícios da semana 4, cobrindo distribuição de probabilidades, inferência bayesiana e testes estatísticos. Cada seção pode ser executada de forma independente.

In [ ]:
import math
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import make_classification, make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.mixture import GaussianMixture
np.random.seed(42)

## 1. Probabilidades básicas e transformações de distribuições

A célula seguinte calcula probabilidades em distribuições Bernoulli e Binomial e mostra como aplicar transformações simples (normalização) em variáveis aleatórias.

In [ ]:
# Probabilidade de sucesso em 3 lançamentos de moeda viciada
p_sucesso = 0.6
n_lancamentos = 3
# Probabilidade de obter exatamente 2 caras
p_2_caras = stats.binom.pmf(2, n_lancamentos, p_sucesso)
# Probabilidade acumulada de obter até 1 cara
p_ate_1_cara = stats.binom.cdf(1, n_lancamentos, p_sucesso)

# Transformação: centralização e normalização de uma variável aleatória simulada
samples = stats.bernoulli.rvs(p_sucesso, size=1_000)
normalized_samples = (samples - samples.mean()) / samples.std()

p_2_caras, p_ate_1_cara, float(normalized_samples.mean()), float(normalized_samples.std())

## 2. Classificador Naïve Bayes (GaussianNB)

Geramos dados sintéticos com duas classes, treinamos um classificador Naïve Bayes Gaussiano e avaliamos a acurácia. Os resultados devem ficar em torno de 85–95%.


In [ ]:
X, y = make_classification(n_samples=1000, n_features=6, n_informative=4, n_redundant=0, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

model = GaussianNB()
model.fit(X_train, y_train)
preds = model.predict(X_test)
accuracy = accuracy_score(y_test, preds)
accuracy

## 3. Modelo de mistura Gaussiana (GMM)

A célula a seguir ajusta um GMM a dados bidimensionais e calcula a pontuação de verossimilhança média por amostra. Essa abordagem pode ser usada para clusterização não supervisionada ou detecção de anomalias.

In [ ]:
X_blobs, _ = make_blobs(n_samples=400, centers=3, cluster_std=0.8, random_state=42)

gmm = GaussianMixture(n_components=3, covariance_type="full", random_state=42)
gmm.fit(X_blobs)
average_log_likelihood = gmm.score(X_blobs)
gmm.means_.round(2), round(average_log_likelihood, 3)

## 4. Simulação de teste A/B

Função utilitária para simular um teste A/B com duas variantes, calculando p-valor do teste z para proporções, intervalo de confiança para a diferença de conversão e uma decisão simples baseada em nível de significância.


In [ ]:
def simulate_ab_test(n_a: int, n_b: int, p_a: float, p_b: float, alpha: float = 0.05):
    """Simula contagens de conversão, calcula p-valor (teste z) e intervalo de confiança.

    Retorna um dicionário com resumo das métricas."""
    # Simular conversões em Bernoulli
    conv_a = np.random.binomial(n_a, p_a)
    conv_b = np.random.binomial(n_b, p_b)

    # Taxas observadas
    rate_a = conv_a / n_a
    rate_b = conv_b / n_b
    diff = rate_b - rate_a

    # Estatística z para diferença de proporções
    p_pool = (conv_a + conv_b) / (n_a + n_b)
    se = math.sqrt(p_pool * (1 - p_pool) * (1 / n_a + 1 / n_b))
    z_score = diff / se if se > 0 else 0.0
    p_value = 2 * (1 - stats.norm.cdf(abs(z_score)))

    # Intervalo de confiança para a diferença
    z_alpha = stats.norm.ppf(1 - alpha / 2)
    ci_low = diff - z_alpha * se
    ci_high = diff + z_alpha * se

    decision = "Rejeita H0 (há evidência de diferença)" if p_value < alpha else "Falha em rejeitar H0"

    return {
        "conversoes": {"A": int(conv_a), "B": int(conv_b)},
        "taxas": {"A": round(rate_a, 4), "B": round(rate_b, 4)},
        "diferenca": round(diff, 4),
        "p_valor": round(p_value, 4),
        "intervalo_confianca": (round(ci_low, 4), round(ci_high, 4)),
        "decisao": decision
    }

# Exemplo de uso: cenário com melhoria de 1 ponto percentual
resultado_ab = simulate_ab_test(n_a=5000, n_b=5000, p_a=0.1, p_b=0.11)
resultado_ab

### Observações
- Ajuste os parâmetros de `simulate_ab_test` para analisar diferentes tamanhos de amostra e efeitos mínimos detectáveis.
- Ao interpretar p-valores, considere também a potência do teste e os custos de falso positivo/falso negativo.
- O intervalo de confiança oferece uma noção de magnitude da diferença, não apenas significância.


## Resolução resumida
- **Binomial:** P(X=2)=C(3,2)·0,6^2·0,4=0,432 e P(X≤1)=0,4^3+3·0,6·0,4^2=0,352. A normalização centraliza a média em ~0 e o desvio padrão em ~1.
- **Naïve Bayes:** o classificador é treinado e avaliado no conjunto de teste, retornando uma acurácia típica entre 0,85–0,95 para o cenário sintético.
- **GMM:** o ajuste recupera três centróides próximos aos centros dos blobs e reporta uma log-verossimilhança média por amostra para avaliar o ajuste.
- **Teste A/B:** o z-teste compara as taxas; p-valor < α implica rejeitar H0 e o IC informa a magnitude do efeito.
